In [1]:
using Pkg; Pkg.activate(".");
using Convex
using SCS

  Activating new project at `~/Dropbox (MIT)/IAP22/6.S098-project`


In [95]:
class_numbers = Dict("6.006"=>1, "6.042"=>2, "6.009"=>3, "6.046"=>4, "6.031"=>5, "18.600"=>6, "6.041"=>7,
"6.008"=>8, "comp_bio"=>9, "NLP"=>10, "comp_vision"=>11, "graphics"=>12, "optimization"=>13, 
    "script_analysis"=>14, "intro_to_acting"=>15, "psych"=>16)

taken = [1 1 1 0 0 1 0 0 0 0 0 0 0 0 0 1]

requirements = [
  ([1], 1),
  ([2], 1),
  ([3], 1),
  ([4], 1),
  ([5], 1),
  ([6, 7, 8], 1),
  ([9, 10, 11, 12, 13], 2),
  ([14, 15, 16], 2)
];

In [96]:
remaining_requirements = []

for (classes, req) in requirements
  classes_left::Array{Int64} = []
  for class in classes
    if taken[class] == 1
      req = max(req - 1, 0)
      classes_left
    else
      push!(classes_left, class)
    end
  end
  if length(classes_left) > 0
    push!(remaining_requirements, (classes_left, req))
  end
end

remaining_requirements

5-element Vector{Any}:
 ([4], 1)
 ([5], 1)
 ([7, 8], 0)
 ([9, 10, 11, 12, 13], 2)
 ([14, 15], 1)

In [97]:
class_names = Dict()
for (name, num) in class_numbers
  class_names[num] = name
end
class_names

Dict{Any, Any} with 16 entries:
  5  => "6.031"
  16 => "psych"
  12 => "graphics"
  8  => "6.008"
  1  => "6.006"
  6  => "18.600"
  11 => "comp_vision"
  9  => "comp_bio"
  14 => "script_analysis"
  3  => "6.009"
  7  => "6.041"
  13 => "optimization"
  15 => "intro_to_acting"
  4  => "6.046"
  2  => "6.042"
  10 => "NLP"

In [98]:
num_classes = length(class_names)
choices = Variable(num_classes)

Variable
size: (16, 1)
sign: real
vexity: affine
id: 900…817

In [99]:
ratings = [
  1,
  0,
  0.5,
  0.25,
  0.01,
  1,
  0,
  0,
  1,
  0.5,
  0.49,
  0,
  0,
  1,
  0,
  0
];

In [100]:
# general 1 unit per class
class_units = ones(num_classes)
obj = -1 * ratings'*choices + class_units'*choices

+ (affine; real)
├─ * (affine; real)
│  ├─ 1×16 adjoint(::Vector{Float64}) with eltype Float64
│  └─ 16-element real variable (id: 900…817)
└─ * (affine; real)
   ├─ 1×16 adjoint(::Vector{Float64}) with eltype Float64
   └─ 16-element real variable (id: 900…817)

In [101]:
# max units allowed (set arbitrarily)
max_units = sum([j for (i, j) in remaining_requirements]) + 1
min_units = 2

consts = [
  #relaxation of binary choice
  choices >= 0,
  choices <= 1,
  # max units
  sum(choices) <= max_units,
  # min units
  sum(choices) >= min_units,
]

# classes already taken cannot be taken again
for (class, done) in enumerate(taken)
  if done == 1
    consts += choices[class] == 0
  end
end

# remaining class requirements
for (classes, req) in remaining_requirements
  req_choices = sum(choices[class] for class in classes)
  consts += req_choices >= req
end

consts

14-element Vector{Constraint}:
 >= constraint (affine)
├─ 16-element real variable (id: 900…817)
└─ 0
 <= constraint (affine)
├─ 16-element real variable (id: 900…817)
└─ 1
 <= constraint (affine)
├─ sum (affine; real)
│  └─ 16-element real variable (id: 900…817)
└─ 6
 >= constraint (affine)
├─ sum (affine; real)
│  └─ 16-element real variable (id: 900…817)
└─ 2
 == constraint (affine)
├─ index (affine; real)
│  └─ 16-element real variable (id: 900…817)
└─ 0
 == constraint (affine)
├─ index (affine; real)
│  └─ 16-element real variable (id: 900…817)
└─ 0
 == constraint (affine)
├─ index (affine; real)
│  └─ 16-element real variable (id: 900…817)
└─ 0
 == constraint (affine)
├─ index (affine; real)
│  └─ 16-element real variable (id: 900…817)
└─ 0
 == constraint (affine)
├─ index (affine; real)
│  └─ 16-element real variable (id: 900…817)
└─ 0
 >= constraint (affine)
├─ index (affine; real)
│  └─ 16-element real variable (id: 900…817)
└─ 1
 >= constraint (affine)
├─ index (affine; real)

In [102]:
problem = minimize(obj, consts)
solve!(problem, SCS.Optimizer())

------------------------------------------------------------------
	       SCS v3.2.1 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 17, constraints m: 45
cones: 	  z: primal zero / dual free vars: 6
	  l: linear vars: 39
settings: eps_abs: 1.0e-04, eps_rel: 1.0e-04, eps_infeas: 1.0e-07
	  alpha: 1.50, scale: 1.00e-01, adaptive_scale: 1
	  max_iters: 100000, normalize: 1, rho_x: 1.00e-06
	  acceleration_lookback: 10, acceleration_interval: 10
lin-sys:  sparse-direct-amd-qdldl
	  nnz(A): 93, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 1.04e+01  1.16e+00  2.17e+01 -9.78e+00  1.00e-01  9.43e-05 
    50| 6.33e-04  1.62e-04  3.24e-04  2.24e+00  1.00e-01  1.52e-04 
--------------------------------------

In [103]:
choices = Int.(round.(choices.value, digits=1)) + taken'

16×1 Matrix{Int64}:
 1
 1
 1
 1
 1
 1
 0
 0
 1
 1
 0
 0
 0
 1
 0
 1

In [104]:
taken_args = [i[2] for i in findall(x->x==1, taken)]
taken_names = [class_names[num] for num in taken_args]

5-element Vector{String}:
 "6.006"
 "6.042"
 "6.009"
 "18.600"
 "psych"

In [105]:
req_names = [([(class_names[i], ratings[i]) for i in classes], req) for (classes, req) in requirements]

8-element Vector{Tuple{Vector{Tuple{String, Float64}}, Int64}}:
 ([("6.006", 1.0)], 1)
 ([("6.042", 0.0)], 1)
 ([("6.009", 0.5)], 1)
 ([("6.046", 0.25)], 1)
 ([("6.031", 0.01)], 1)
 ([("18.600", 1.0), ("6.041", 0.0), ("6.008", 0.0)], 1)
 ([("comp_bio", 1.0), ("NLP", 0.5), ("comp_vision", 0.49), ("graphics", 0.0), ("optimization", 0.0)], 2)
 ([("script_analysis", 1.0), ("intro_to_acting", 0.0), ("psych", 0.0)], 2)

In [106]:
chosen = []
for (num, choice) in enumerate(choices)
  if choice == 1
    push!(chosen, class_names[num])
  end
end
chosen

10-element Vector{Any}:
 "6.006"
 "6.042"
 "6.009"
 "6.046"
 "6.031"
 "18.600"
 "comp_bio"
 "NLP"
 "script_analysis"
 "psych"